In [1]:
from serving_client import ServingClient
from game_client import GameClient

import ipywidgets as widgets
from IPython.display import display, clear_output, Markdown
from prettytable import PrettyTable
import numpy as np

In [3]:
# sc = ServingClient(ip='serving') # port = 8890
sc = ServingClient()
sc.logs()

## Download model

### Hello David: please use below two models
- (default model):{'workspace': 'xiaoxin-zhou', \
                    'model': 'decision-tree-approach-2', \ 
                    'version': '1.0.0'}
- (XGBoost model):{'workspace': 'xiaoxin-zhou', \
                    'model': 'xgboost-5-2', \ 
                    'version': '1.0.3'}

In [4]:
workspace = widgets.Text(placeholder='Workspace')
model = widgets.Text(placeholder='Model')
version = widgets.Text(placeholder='Version')

button_send = widgets.Button(
                description='Download model',
                tooltip='Send',
                style={'description_width': 'initial'}
            )

output = widgets.Output()
def on_button_clicked(event):
    with output:
        clear_output()
        print("Model information: ")
        print(f"Model: {model.value}\nVersion: {version.value}")

        try:
            sc.download_registry_model(workspace='xiaoxin-zhou',\
                                       model=model.value,\
                                       version=version.value)
        except Exception as e:
            print(f"Failed to download model{model.value} with version {version.value}.\nError:{e}")
        print(f"Successfully downloaded model {model.value} with version {version.value}.")
        
        
button_send.on_click(on_button_clicked)
vbox_result = widgets.VBox([workspace, model, version, button_send, output])
vbox_result

## Ping Game

In [5]:
# copied from m3_q4_ping_game.ipynb

game_id_input = widgets.Text(
       value='2021020329',
       description='Game ID', )

button = widgets.Button(description='Ping Game')
out = widgets.Output()

gc = GameClient()

def on_button_clicked(_):
    # "linking function with output"
    with out:# what happens when we press the button
        clear_output()
        
        int_game_id = int(game_id_input.value)
        
        print(f'pinging game: {game_id_input.value}')

        # process and predict data
        df_without_team_name = gc.process_and_predict_data(int_game_id)
        print(f'df types: {df_without_team_name.dtypes}')
        xG = sc.predict(df_without_team_name)

        gc.add_pred_probs_to_df(int_game_id, xG)
        # display dataframe of sum of xGs
        df_sum_xG = gc.get_xG_table(int_game_id)
        print(df_sum_xG)
        
        # display new_df of all features
        df_all = gc.games_dataframes[int_game_id]
        print(df_all)
        
            
# linking button and function together using a button's method
button.on_click(on_button_clicked)

# displaying button and its output together
widgets.VBox([game_id_input,button,out])